In [7]:
import pandas as pd
import numpy as np
from neuralprophet import NeuralProphet
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings(category=FutureWarning, action="ignore")

In [8]:
NUM_PV = 1

pv_dfs = [
    pd.read_csv(f"../../data/renewable/renewable{i+1}.csv.gz", compression='gzip', index_col=0)
    for i in range(1)
]

price_df = pd.read_csv("data/price.csv")
price_df = price_df.rename(columns={"TIME": "ds", "PRICE": "y"})
price_df['ds'] = pd.to_datetime(price_df['ds'])
price_df = price_df.iloc[:pv_dfs[0].shape[0]]

for i in range(len(pv_dfs)):
    pv_df = pv_dfs[i]
    pv_df["ds"] = price_df["ds"]
    pv_df = pv_df[["ds", "0"]].rename(columns={"0": "y"})
    pv_dfs[i] = pv_df

print(price_df.head())
print(price_df.shape)

print(pv_dfs[0].head())
print(pv_dfs[0].shape)


                   ds      y
0 2024-01-01 00:00:00  63.33
1 2024-01-01 00:15:00  60.02
2 2024-01-01 00:30:00  56.71
3 2024-01-01 00:45:00  53.40
4 2024-01-01 01:00:00  50.09
(8760, 2)
                   ds    y
0 2024-01-01 00:00:00  0.0
1 2024-01-01 00:15:00  0.0
2 2024-01-01 00:30:00  0.0
3 2024-01-01 00:45:00  0.0
4 2024-01-01 01:00:00  0.0
(8760, 2)


In [9]:
METRICS = ["MAE", "RMSE"]
METRICS_VAL = ["MAE_val", "RMSE_val"]

def cv_train(df, lags, forecasts):
    folds = NeuralProphet(n_lags=lags, n_forecasts=forecasts)\
        .crossvalidation_split_df(df, freq='15min', k=10)

    metrics_train = pd.DataFrame(columns=METRICS)
    metrics_test = pd.DataFrame(columns=METRICS_VAL)

    n_fold = 1
    for df_train, df_test in folds:
        print(f"Fold {n_fold}/{len(folds)}")
        n_fold += 1
        m = NeuralProphet(n_lags=lags, n_forecasts=forecasts, batch_size=32)
        train = m.fit(df=df_train, freq="15min", validation_df=df_test)
        test = m.test(df=df_test)
        metrics_train = metrics_train.append(train[METRICS].iloc[-1])
        metrics_test = metrics_test.append(test[METRICS_VAL].iloc[-1])

    print(metrics_test.describe().loc[["mean", "std", "min", "max"]])

    return metrics_train, metrics_test

def train(df, lags, forecasts, split=0.67):
    df_train, df_test = train_test_split(df, test_size=split, shuffle=False)
    m = NeuralProphet(n_lags=lags, n_forecasts=forecasts, batch_size=32, epochs=1)
    train_res = m.fit(df=df_train, freq="15min", validation_df=df_test)
    return m, df_train, df_test, train_res

def predict(m, df_train, df_test):
    future = m.make_future_dataframe(df=df_train, periods=len(df_test))
    forecast = m.predict(future)
    return forecast

In [10]:
m, df_train, df_test, train_res = train(
    df=price_df,
    lags=24,
    forecasts=24
)

WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
INFO - (NP.df_utils._infer_frequency) - Major frequency 15T corresponds to 99.965% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - 15min
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO - (NP.utils.set_auto_seasonalities) - Disabling yearly seasonality. Run NeuralProphet with yearly_seasonality=True to override this.
WARNING - (NP.config.set_lr_finder_args) - Learning rate finder: The number of batches (89) is too small than the required number for the learning rate finder (236). The results might not be optimal.


Finding best initial lr:   0%|          | 0/236 [00:00<?, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [11]:
df_train = df_train.tail(48)
m.predict(df_train)

INFO - (NP.df_utils._infer_frequency) - Major frequency 15T corresponds to 97.917% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - 15min
INFO - (NP.df_utils._infer_frequency) - Major frequency 15T corresponds to 98.611% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - 15min


Predicting: 89it [00:00, ?it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column


,ds,y,yhat1,yhat2,yhat3,yhat4,yhat5,yhat6,yhat7,yhat8,...,ar18,ar19,ar20,ar21,ar22,ar23,ar24,trend,season_weekly,season_daily
0,2024-01-30 14:30:00,71.9400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-01-30 14:45:00,71.9450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-01-30 15:00:00,71.9500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-01-30 15:15:00,73.6850,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-01-30 15:30:00,75.4200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2024-01-30 15:45:00,77.1550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2024-01-30 16:00:00,78.8900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2024-01-30 16:15:00,81.3750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2024-01-30 16:30:00,83.8600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2024-01-30 16:45:00,86.3450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
from neuralprophet.utils import save

save(m, "price_predictor.np")